In [1]:
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import random
import torch
import matplotlib.pyplot as plt
# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.utils.visualizer import VisImage
from detectron2.data import MetadataCatalog

In [2]:
im = cv2.imread("input.jpg")
print(im.shape)
cfg = get_cfg()
# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
cfg.merge_from_file(model_zoo.get_config_file("../configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
cfg.merge_from_list(['MODEL.DEVICE','cpu'])
# Find a model from detectron2's model zoo. You can either use the https://dl.fbaipublicfiles.... url, or use the detectron2:// shorthand
cfg.MODEL.WEIGHTS = "detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl"
predictor = DefaultPredictor(cfg)
outputs = predictor(im)

(480, 640, 3)
WARNING [01/07 21:43:38 d2.config.compat]: Config '/home/asung/detectron2/detectron2/detectron2/detectron2/model_zoo/configs/../configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml' has no VERSION. Assuming it to be compatible with latest v2.


In [3]:
# cv2.imshow('image',im)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [74]:
# look at the outputs. See https://detectron2.readthedocs.io/tutorials/models.html#model-output-format for specification
#print(outputs)
#print("\n\n\n\n\n\n")
print(outputs["instances"],"\n\n")
print("class: ",outputs["instances"].pred_classes.shape,"\n",outputs["instances"].pred_classes,"\n")
print("boxes: ",outputs["instances"].pred_boxes.tensor.shape,"\n",outputs["instances"].pred_boxes,"\n")
print("scores: " ,outputs["instances"].scores.shape ,"\n",outputs["instances"].scores,"\n")
print("masks:\n",type(outputs["instances"].pred_masks),outputs["instances"].pred_masks.shape,"\n")

Instances(num_instances=15, image_height=480, image_width=640, fields=[pred_boxes, scores, pred_classes, pred_masks]) 


class:  torch.Size([15]) 
 tensor([17,  0,  0,  0,  0,  0,  0,  0, 25,  0, 25, 25,  0,  0, 24]) 

boxes:  torch.Size([15, 4]) 
 Boxes(tensor([[126.6035, 244.8977, 459.8291, 480.0000],
        [251.1083, 157.8127, 338.9731, 413.6379],
        [114.8496, 268.6864, 148.2351, 398.8111],
        [  0.8217, 281.0327,  78.6072, 478.4210],
        [ 49.3954, 274.1229,  80.1545, 342.9808],
        [561.2249, 271.5816, 596.2755, 385.2552],
        [385.9072, 270.3125, 413.7130, 304.0397],
        [515.9295, 278.3744, 562.2792, 389.3803],
        [335.2410, 251.9167, 414.7491, 275.9375],
        [350.9300, 269.2060, 386.0984, 297.9081],
        [331.6292, 230.9996, 393.2759, 257.2009],
        [510.7349, 263.2656, 570.9865, 295.9194],
        [409.0841, 271.8646, 460.5582, 356.8722],
        [506.8766, 283.3257, 529.9404, 324.0392],
        [594.5663, 283.4820, 609.0577, 311.41

In [132]:
poslist = []
boxes = outputs["instances"].pred_boxes.tensor
pred = outputs['instances'].pred_classes
masks = outputs["instances"].pred_masks
for i in range(outputs['instances'].scores.shape[0]):
    if pred[i] == 0: 
        wide = abs(boxes[i][2] - boxes[i][0])*abs(boxes[i][3]-boxes[i][1])
        poslist.append(int(wide))
    else:
        poslist.append(0)

print(poslist)
idx = poslist.index(max(poslist))

x_s,y_s,x_d,y_d = boxes[idx]

[0, 22478, 4344, 15353, 2118, 3984, 937, 5145, 0, 1009, 0, 0, 4375, 939, 0]


In [133]:
def binarysearch(mask,x_s,x_d,flag):
    maskArry = torch.where(mask == True)[0] 
    if maskArry.size() == torch.Size([0]):
        return -1
    if flag :
        src = min(maskArry)+1
    else : src = max(maskArry)-1
    return src
    

def expand(masks,size,y_s,y_d,x_s,x_d):
    mask = masks.clone()
    masktemp = masks.clone()
    y_s,y_d,x_s,x_d = int(y_s),int(y_d),int(x_s),int(x_d)
    for i in range(y_s,y_d):
        leftx = binarysearch(masktemp[i],x_s,x_d,True)
        rightx = binarysearch(masktemp[i],x_s,x_d,False)
        if leftx == -1 :
            continue
        #print(leftx,rightx)
        for j in range(size):
            mask[i][leftx-j] = True    #왼쪽으로 쭉          
            mask[i][rightx+j] = True
            
    masktemp = masks.clone()
    for i in range(x_s,x_d):
        upy = binarysearch(masktemp.T[i],y_s,y_d,True)
        downy = binarysearch(masktemp.T[i],y_s,y_d,False)
        if upy == -1 :
            continue
        for j in range(size):
            mask[upy-j][i] = True    #위로 쭉
            mask[downy+j][i] = True
    return mask
    


# We can use `Visualizer` to draw the predictions on the image.
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.0)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
print(type(v))
#                   box(x_s,y_s,x_d,y_d)
#                   img[y_s:y_d, x_s:x_d]
vtmp = v.get_image()[:,:,::-1]#[244:480,126:459,::-1]
print(type(vtmp))

tmp = im[int(y_s):int(y_d),int(x_s):int(x_d)] #네모 박스 칸만큼 slie


tim = torch.from_numpy(im.copy())  #tim은  im의 복사본, tensor로 변환

be = torch.where(masks[idx]!=True)
    
#remove background
tim[be] = 224

tim = tim.numpy()
print(be)
#tim = tim[int(y_s):int(y_d),int(x_s):int(x_d)]
print(tim[be])
def fitsize(im,y_s,y_d,x_s,x_d):
    y_s,y_d,x_s,x_d = int(y_s),int(y_d),int(x_s),int(x_d)
    return im[y_s:y_d,x_s:x_d]

<class 'detectron2.utils.visualizer.VisImage'>
<class 'numpy.ndarray'>
(tensor([  0,   0,   0,  ..., 479, 479, 479]), tensor([  0,   1,   2,  ..., 637, 638, 639]))
[[224 224 224]
 [224 224 224]
 [224 224 224]
 ...
 [224 224 224]
 [224 224 224]
 [224 224 224]]


In [134]:
etim = torch.from_numpy(im.copy())
emask = expand(masks[idx],20,y_s,y_d,x_s,x_d)
ebe = torch.where(emask!=True)
etim[ebe] = 224
etim = etim.numpy()
etim.shape

(480, 640, 3)

In [135]:
gauss = cv2.GaussianBlur(im,(5,5),1e+10)
median = cv2.medianBlur(im,5)
tr = torch.where(masks[idx] == True)
blur[tr] = im[tr]
median[tr] = im[tr]
blur = fitsize(blur,int(y_s),int(y_d),int(x_s),int(x_d))
median = fitsize(median,int(y_s),int(y_d),int(x_s),int(x_d))

# 배경자르기..
cv2.imshow('be',tim)
cv2.imshow('ori',tmp)
cv2.imshow('imagin',blur)
cv2.imshow('median',median)
cv2.imshow('test',etim)
cv2.waitKey(0)
cv2.destroyAllWindows()